In [ ]:
import pandas as pd
import json
import glob
import os
import numpy as np
from datetime import datetime
import sys

# --- Setup Paths ---
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
SRC_PATH = os.path.join(PROJECT_ROOT, "src")
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

# ============================================================
# Safe Parsing Helpers (per specification)
# ============================================================
def safe_get(d, *keys, default='N/A'):
    """Safe nested dictionary access with default fallback"""
    for key in keys:
        if isinstance(d, dict):
            d = d.get(key, {})
        else:
            return default
    return d if (d != {} and d is not None) else default

def safe_float(x, default=0.0):
    """Convert to float safely, strip %, handle None/N/A"""
    if x is None or x == 'N/A':
        return default
    try:
        if isinstance(x, str):
            x = x.replace('%', '').strip()
        return float(x)
    except:
        return default

# ============================================================
# File Discovery (exact suffix matching per specification)
# ============================================================
DERIV_ROOT = os.path.join(PROJECT_ROOT, "derivatives")

# Required suffixes for exact matching (updated to match actual file names)
REQUIRED_SUFFIXES = [
    "__step01_loader_report.json",  # step_01 (actual naming)
    "__preprocess_summary.json",    # step_02  
    "__filtering_summary.json",     # step_04
    "__reference_summary.json",     # step_05
    "__kinematics_summary.json"     # step_06
]

# Scan recursively for *.json files
json_files = glob.glob(os.path.join(DERIV_ROOT, "**", "*.json"), recursive=True)

# Filter files with exact suffix matching
valid_files = []
for json_path in json_files:
    filename = os.path.basename(json_path)
    if any(filename.endswith(suffix) for suffix in REQUIRED_SUFFIXES):
        valid_files.append(json_path)

print(f"📁 Found {len(valid_files)} valid JSON files out of {len(json_files)} total")

# ============================================================
# Load + Group by Run_ID (per specification)
# ============================================================
from collections import defaultdict
runs = defaultdict(dict)

for json_path in valid_files:
    filename = os.path.basename(json_path)
    run_id = filename.split("__")[0]
    
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except Exception as e:
        print(f"⚠️  Failed to load {filename}: {e}")
        continue
    
    # Categorize by exact suffix matching
    if filename.endswith("__step01_loader_report.json"):
        runs[run_id]["step_01"] = data
    elif filename.endswith("__preprocess_summary.json"):
        runs[run_id]["step_02"] = data
    elif filename.endswith("__filtering_summary.json"):
        runs[run_id]["step_04"] = data
    elif filename.endswith("__reference_summary.json"):
        runs[run_id]["step_05"] = data
    elif filename.endswith("__kinematics_summary.json"):
        runs[run_id]["step_06"] = data

# Skip runs missing critical data (require step_01 and step_06)
complete_runs = {rid: steps for rid, steps in runs.items() 
                if steps.get('step_01') and steps.get('step_06')}

print(f"📊 Found data for {len(complete_runs)} complete run(s)")
print(f"📁 Steps loaded per run:")
for rid, steps in complete_runs.items():
    print(f"  {rid}: {list(steps.keys())}")
print()

📊 Found data for 3 run(s)
📁 Steps loaded per run:
  734_T1_P1_R1_Take 2025-12-01 02.18.27 PM: ['step_06', 'step_04', 'step_02', 'step_01', 'step_05']
  734_T1_P2_R1_Take 2025-12-01 02.28.24 PM: ['step_06', 'step_04', 'step_02', 'step_01', 'step_05']
  763_T2_P2_R2_Take_2025-12-25 10.51.23 AM_005: ['step_06', 'step_04', 'step_02', 'step_01', 'step_05']



ModuleNotFoundError: No module named 'xlsxwriter'

In [ ]:
# ============================================================
# Master Row Schema (fixed columns per specification)
# ============================================================
all_summaries = []

for run_id, steps in complete_runs.items():
    s01 = steps.get('step_01', {})
    s02 = steps.get('step_02', {})
    s04 = steps.get('step_04', {})
    s05 = steps.get('step_05', {})
    s06 = steps.get('step_06', {})
    
    # ============================================================
    # Canonical Fields (normalize differences per specification)
    # ============================================================
    
    # Sampling rate: fps = first_available with fallback to 120.0
    fps = safe_float(
        safe_get(s01, 'raw_data_quality', 'sampling_rate_actual',
                'fs_actual_hz', 'sampling_rate_hz'),
        default=120.0
    )
    
    # Reference status: normalize to uppercase
    ref_status_raw = safe_get(s05, 'reference_metrics', 'ref_quality_status', default='MISSING')
    ref_status = str(ref_status_raw).upper()
    
    # ============================================================
    # Gap Units Logic (use ms directly when available)
    # ============================================================
    max_gap_frames = safe_get(s02, 'max_interpolation_gap', default=0)
    max_gap_ms_raw = safe_get(s02, 'max_gap_ms')  # Check if ms is directly available
    
    if max_gap_ms_raw != 'N/A' and max_gap_ms_raw is not None:
        # Use ms directly when available
        max_gap_ms = safe_float(max_gap_ms_raw)
    else:
        # Compute ms from frames only if ms is missing
        max_gap_ms = round((safe_float(max_gap_frames) / fps) * 1000, 2)
    
    # ============================================================
    # Build Master Row
    # ============================================================
    row = {
        # --- Identity ---
        "Run_ID": run_id,
        "Processing_Date": safe_get(s01, 'identity', 'processing_timestamp'),
        
        # --- Step01 Fields ---
        "OptiTrack_Error_mm": safe_float(safe_get(s01, 'raw_data_quality', 'optitrack_mean_error_mm')),
        "Total_Frames": safe_get(s01, 'raw_data_quality', 'total_frames', default=0),
        
        # --- Step02 Fields ---
        "Missing_Raw_%": safe_float(safe_get(s02, 'raw_missing_percent')),
        "Max_Gap_Frames": max_gap_frames,
        "Max_Gap_MS": max_gap_ms,
        "Bone_Stability_CV": safe_float(safe_get(s02, 'bone_qc_mean_cv')),
        "Skeletal_Alerts": safe_get(s02, 'bone_qc_alerts', default=0),
        "Worst_Bone": safe_get(s02, 'worst_bone'),
        
        # --- Step05 Fields ---
        "Ref_Stability_mm": safe_float(safe_get(s05, 'reference_metrics', 'ref_stability_mm')),
        "Ref_Status": ref_status,
        
        # --- Step06 Signal Quality ---
        "Signal_Noise_RMS": safe_float(safe_get(s06, 'signal_quality', 'avg_vel_residual_rms')),
        "Dom_Freq_Hz": safe_float(safe_get(s06, 'signal_quality', 'avg_dominant_freq_hz')),
        "Quat_Norm_Error": safe_float(safe_get(s06, 'signal_quality', 'max_quat_norm_error')),
        
        # --- Step06 Kinematics ---
        "Max_Ang_Vel": safe_float(safe_get(s06, 'metrics', 'angular_velocity', 'max')),
        "Mean_Ang_Vel": safe_float(safe_get(s06, 'metrics', 'angular_velocity', 'mean')),
        "Max_Lin_Acc": safe_float(safe_get(s06, 'metrics', 'linear_accel', 'max')),
        "Outlier_Frames": safe_get(s06, 'effort_metrics', 'outlier_frame_count', default=0),
        
        # --- Step06 Effort Metrics ---
        "Path_Length_M": round(safe_float(safe_get(s06, 'effort_metrics', 'total_path_length_mm')) / 1000, 2),
        "Intensity_Index": safe_float(safe_get(s06, 'effort_metrics', 'intensity_index')),
        
        # --- Overall Status ---
        "Pipeline_Status": safe_get(s06, 'overall_status'),
    }
    
    # ============================================================
    # Quality Scoring (labeled heuristic per specification)
    # ============================================================
    score = 100.0
    
    # Penalties for data quality
    score -= safe_float(row["Missing_Raw_%"]) * 5
    score -= (safe_float(row["Max_Gap_MS"]) / 10)  # Penalty for large gaps
    
    # Penalties for skeletal stability
    score -= safe_float(row["Bone_Stability_CV"]) * 10 
    score -= safe_float(row["Skeletal_Alerts"]) * 5
    
    # Penalty for reference stability
    ref_stab = safe_float(row["Ref_Stability_mm"])
    if ref_stab > 4.0: 
        score -= 15
    
    # Penalty for signal quality issues
    if safe_float(row["Quat_Norm_Error"]) > 0.1:
        score -= 10
    
    row["Quality_Score"] = round(max(0, min(100, score)), 2)
    row["Quality_Score_Method"] = "heuristic_v1"  # Required label per specification
    
    # ============================================================
    # Research Decision Rule (deterministic per specification)
    # ============================================================
    if (row["Pipeline_Status"] == "PASS" and 
        row["Quality_Score"] >= 75 and 
        row["Ref_Status"] == "PASS" and 
        safe_float(row["Bone_Stability_CV"]) < 1.5):
        row["Research_Decision"] = "ACCEPT"
    elif row["Pipeline_Status"] == "PASS" and row["Quality_Score"] >= 50:
        row["Research_Decision"] = "REVIEW"
    else:
        row["Research_Decision"] = "REJECT"
    
    all_summaries.append(row)

print(f"📊 Processed {len(all_summaries)} complete runs")

In [ ]:
# ============================================================
# Export to Excel (per specification)
# ============================================================
if not all_summaries:
    print("❌ No complete runs found to aggregate!")
else:
    df_master = pd.DataFrame(all_summaries)
    df_master = df_master.sort_values('Quality_Score', ascending=False).reset_index(drop=True)
    
    REPORTS_DIR = os.path.join(PROJECT_ROOT, "reports")
    os.makedirs(REPORTS_DIR, exist_ok=True)
    excel_path = os.path.join(REPORTS_DIR, f"Master_Audit_Log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx")
    
    with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
        df_master.to_excel(writer, index=False, sheet_name='Audit_Log')
        workbook = writer.book
        worksheet = writer.sheets['Audit_Log']
        
        # Header styling
        header_fmt = workbook.add_format({
            'bold': True, 
            'bg_color': '#4472C4', 
            'font_color': 'white',
            'text_wrap': True
        })
        for col_num, value in enumerate(df_master.columns.values):
            worksheet.write(0, col_num, value, header_fmt)
        
        # Conditional formatting for Research_Decision (green/yellow/red)
        red_fmt = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})
        yellow_fmt = workbook.add_format({'bg_color': '#FFEB9C', 'font_color': '#9C6500'})
        green_fmt = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'})
        
        col_idx = df_master.columns.get_loc("Research_Decision")
        for row_num in range(1, len(df_master) + 1):
            decision = df_master.iloc[row_num-1]['Research_Decision']
            if decision == 'ACCEPT':
                worksheet.write(row_num, col_idx, decision, green_fmt)
            elif decision == 'REVIEW':
                worksheet.write(row_num, col_idx, decision, yellow_fmt)
            else:
                worksheet.write(row_num, col_idx, decision, red_fmt)
        
        # Auto-fit column widths capped at 40 characters
        for i, col in enumerate(df_master.columns):
            max_len = max(
                df_master[col].astype(str).str.len().max(),
                len(str(col))
            )
            worksheet.set_column(i, i, min(max_len + 2, 40))
    
    print(f"\n{'='*70}")
    print(f"🎉 Master Audit Log Created")
    print(f"{'='*70}")
    print(f"📊 Total Runs: {len(all_summaries)}")
    print(f"💾 File: {excel_path}")
    print(f"{'='*70}\n")
    
    print("Decision Summary:")
    print(df_master['Research_Decision'].value_counts())
    
    print("\nQuality Score Stats:")
    print(f"  Mean: {df_master['Quality_Score'].mean():.2f}")
    print(f"  Min:  {df_master['Quality_Score'].min():.2f}")
    print(f"  Max:  {df_master['Quality_Score'].max():.2f}")
    
    print("\nQuality Score Method:")
    print(f"  {df_master['Quality_Score_Method'].iloc[0] if len(df_master) > 0 else 'N/A'}")
    
    print("\nPreview:")
    display(df_master.head())